**Sentence Completion using GPT2**

In [4]:
!pip3 install pytorch-transformers

     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 860kB 39.4MB/s 
     |████████████████████████████████| 655kB 39.7MB/s 
     |████████████████████████████████| 1.0MB 30.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=ed6ed2dd0c885f660927a05a0c15e016fd373c3238c6715b95ff0fe2e3d03450
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609233 sha256=c37dd2d4521d9635dee45f203226f06b7cfce06dd37e79bec10d769abf3c8c4a
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built sacremoses regex


In [0]:
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [26]:
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

100%|██████████| 548118077/548118077 [00:11<00:00, 48989218.67B/s]


**One word completion**

In [77]:
text = "who is your"
indexed_tokens = tokeniser.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
model.eval() # Setting the model in evaluation mode to deactivate the DropOut modules
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [0]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

In [79]:
print(outputs[0].shape)
print(predictions)

torch.Size([1, 3, 50257])
tensor([[[ -41.9926,  -42.0111,  -45.8861,  ...,  -47.4671,  -46.0255,
           -41.8869],
         [-118.9843, -117.2208, -123.5588,  ..., -123.6198, -122.7697,
          -120.4119],
         [-105.8518, -104.3982, -108.1491,  ..., -106.3690, -109.4703,
          -104.2819]]], device='cuda:0')


In [0]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokeniser.decode(indexed_tokens + [predicted_index])

In [81]:
print(predicted_text)

 who is your friend


**Muliple Word Completion**

In [86]:
chars = 0
text = "who is responsible"
while chars<20:
  chars += 1
  indexed_tokens = tokeniser.encode(text)
  tokens_tensors = torch.tensor([indexed_tokens])
  tokens_tensors = tokens_tensors.to('cuda')
#  model = model.to('cuda')
  with torch.no_grad():
    outputs = model(tokens_tensors)
    predictions = outputs[0]
  predicted_index = torch.argmax(predictions[0,-1,:]).item()
  text = tokeniser.decode(indexed_tokens + [predicted_index])

print(text)

 who is responsible for the development of the new technology.



**Text Generation**

In [15]:
!pip3 install transformers

     |████████████████████████████████| 296kB 3.5MB/s 


In [1]:
!git clone https://github.com/huggingface/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 9922, done.
remote: Total 9922 (delta 0), reused 0 (delta 0), pack-reused 9922
Receiving objects: 100% (9922/9922), 5.08 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (7232/7232), done.


In [13]:
!python pytorch-transformers/examples/run_generation.py 

Traceback (most recent call last):
  File "pytorch-transformers/examples/run_generation.py", line 29, in <module>
    from transformers import GPT2Config, OpenAIGPTConfig, XLNetConfig, TransfoXLConfig, XLMConfig
ModuleNotFoundError: No module named 'transformers'


In [23]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=100 \
    --model_name_or_path=gpt2 \

10/06/2019 20:39:59 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
10/06/2019 20:39:59 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /root/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
10/06/2019 20:39:59 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
10/06/2019 20:39:5

In [24]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=xlnet \
    --length=100 \
    --model_name_or_path=xlnet-base-cased \

10/06/2019 20:42:09 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model from cache at /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8
10/06/2019 20:42:09 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.ef1824921bc0786e97dc88d55eb17aabf18aac90f24bd34c0650529e7ba27d6f
10/06/2019 20:42:09 - INFO - transformers.configuration_utils -   Model config {
  "attn_type": "bi",
  "bi_data": false,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "initializer_range": 0.02,
 

In [25]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=transfo-xl \
    --length=100 \
    --model_name_or_path=transfo-xl-wt103 \

10/06/2019 20:43:02 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin from cache at /root/.cache/torch/transformers/b24cb708726fd43cbf1a382da9ed3908263e4fb8a156f9e0a4f45b7540c69caa.a6a9c41b856e5c31c9f125dd6a7ed4b833fbcefda148b627871d4171b25cffd1
10/06/2019 20:43:02 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-config.json from cache at /root/.cache/torch/transformers/a6dfd6a3896b3ae4c1a3c5f26ff1f1827c26c15b679de9212a04060eaf1237df.aef76fb1064c932cd6a2a2be3f23ebbfa5f9b6e29e8e87b571c45b4a5d5d1b90
10/06/2019 20:43:02 - INFO - transformers.configuration_utils -   Model config {
  "adaptive": true,
  "attn_type": 0,
  "clamp_len": 1000,
  "cutoffs": [
    20000,
    40000,
    200000
  ],
  "d_embed": 1024,
  "d_head": 64,
  "d_inner": 4096,
  "d_model": 1024,
  "div_val": 4,
  "dropatt": 0.0,
  "dropout": 0.1,